In [ ]:
#This code is for tagging the types of document, specifically, using k-means to cluster and tag each doc with its cluster.

In [1]:
#mount gdrive. notice: the dataset should be uploaded already
from google.colab import drive
import os
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn import feature_extraction  

In [3]:
questions=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/cutquestions.csv', lineterminator='\n')

在斷詞完成後，有些許內容為空值，這類資料在fit模型時會造成錯誤，因此以「無」代替


In [4]:
#first preprocess the empty value and nan value
for i in range(len(questions)):
  
  #for those empty content, assign a '無' string to avoid nan error
  if type(questions['content'][i])==str:
    if questions['content'][i].strip()=='':
      questions['content'][i]='無'
  elif type(questions['content'][i])!=str:
    questions['content'][i]='無'


在輸入KNN模型前，須先將各文本轉換成TFIDF document vector


In [5]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()
doc_vec=transformer.fit_transform(vectorizer.fit_transform(questions['content']))

#transform the data into tfidf document vector
tfidf_model = TfidfVectorizer().fit_transform(questions['content'])
word = vectorizer.get_feature_names()
weight = doc_vec.toarray()

In [6]:
#use K-means to tag five clusters
from sklearn.cluster import KMeans
clf = KMeans(n_clusters=5)
s = clf.fit(weight)

In [27]:
clf.labels_[0:10]

array([2, 2, 2, 1, 4, 2, 2, 2, 0, 2], dtype=int32)

In [10]:
# for each document, assign a label
questions['tag']=0
for i in range(len(questions['content'])):
  questions['tag'][i]=clf.labels_[i]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [34]:
#save the tagged df
questions.to_csv('/content/gdrive/My Drive/Colab Notebooks/questions_w_tag.csv',encoding='utf-8',index=False)

In [12]:
#reload the data (not necessary, just to check if the saved data is well tagged)
questions=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/questions_w_tags.csv', lineterminator='\n')
behaviors=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/behaviors.csv', lineterminator='\n')


In [31]:
#tag each behavior with the document tag 
behaviors['tag']=0
for i in range(len(behaviors['pid'])):
  behaviors['tag'][i]=questions.loc[questions['question_id']==behaviors['question_id'][i]].tag


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [33]:
#save the behaviors after processing
behaviors.to_csv('/content/gdrive/My Drive/Colab Notebooks/behaviors_w_tag.csv',encoding='utf-8',index=False)

In [15]:
#as long as we have each tag of the document which users had read/shared/...etc, we can know the preference/favor of the user.
behaviors.head()

,pid,question_id,event,event_date,tag
0,00000000-0000-0000-0000-000000000000,2538e41b-7336-4d81-8e2e-a27925c00161,view,2020-09-01 00:00:06,1
1,00000000-0000-0000-0000-000000000000,f4d476c5-f3e1-4bd3-9d56-bf3db0f12853,view,2020-09-01 00:00:40,4
2,00000000-0000-0000-0000-000000000000,ae67939b-38f0-48eb-86e9-878c57aa3759,view,2020-09-01 00:00:41,1
3,2384cc6e-2389-4be1-bc0d-84d682ea2714,c3c59ddc-1b88-4241-a0d3-f6161ea0bd0d,view,2020-09-01 00:00:57,1
4,cc1e6dfa-4253-4919-8922-51667e7e39be,2538e41b-7336-4d81-8e2e-a27925c00161,view,2020-09-01 00:01:13,1


In [32]:
behaviors['tag'][250242]

1

In [28]:
questions.loc[questions['question_id']==behaviors['question_id'][250242]].tag

3839    1
Name: tag, dtype: int64